In [1]:
import pytest
import joblib
import tensorflow as tf
import os

def test_load_xgb_model():
    # Verificar que el archivo del modelo exista
    model_path = 'src/model/xgboost_model.joblib'
    scaler_path = 'src/model/xgb_scaler.joblib'
    assert os.path.exists(model_path), f"{model_path} no encontrado"
    assert os.path.exists(scaler_path), f"{scaler_path} no encontrado"

    # Cargar el modelo y el escalador
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)

    # Verificar que el modelo y el escalador no sean None
    assert model is not None
    assert scaler is not None

def test_load_nn_model():
    model_path = 'src/model/nn_stroke_model.keras'
    scaler_path = 'src/model/nn_scaler.joblib'
    assert os.path.exists(model_path), f"{model_path} no encontrado"
    assert os.path.exists(scaler_path), f"{scaler_path} no encontrado"

    # Cargar el modelo y el escalador
    model = tf.keras.models.load_model(model_path)
    scaler = joblib.load(scaler_path)

    # Verificar que el modelo y el escalador no sean None
    assert model is not None
    assert scaler is not None


In [2]:
import pytest
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def test_preprocess_data():
    # Datos de ejemplo con el mismo formato que los datos de entrenamiento
    data = {
        'gender': [0, 1],
        'hypertension': [0, 0],
        'heart_disease': [0, 1],
        'ever_married': [1, 0],
        'work_type': [0.059, -2.95],
        'Residence_type': [1, 0],
        'smoking_status': [-0.24, 0.55],
        'bmi_category': [0.17, -2.81],
        'age_category': [-3.92, 1.06],
        'glucose_level_category': [0.92, -0.30],
        'stroke': [0, 1]
    }
    df = pd.DataFrame(data)
    
    # Escalador estándar
    scaler = StandardScaler()
    X = df.drop(columns=['stroke']).values
    y = df['stroke'].values

    # Escalado de las características
    X_scaled = scaler.fit_transform(X)

    # Verificar el tamaño y formato de las salidas
    assert X_scaled.shape == (2, 10), "El tamaño del conjunto de características no es el esperado"
    assert isinstance(X_scaled, np.ndarray), "La salida del escalado no es un array de numpy"
    assert y.shape == (2,), "El tamaño del vector de etiquetas no es el esperado"


In [3]:
import pytest
import pandas as pd
import numpy as np
from unittest.mock import patch
from src.model.train_models_XGBoost import XGBoostStrokeModel
import tensorflow as tf
import joblib

@pytest.fixture
def sample_input_data():
    # Datos de prueba con formato similar al que la aplicación espera recibir
    return pd.DataFrame({
        'gender': [1],
        'hypertension': [0],
        'heart_disease': [0],
        'ever_married': [1],
        'work_type': [0],
        'Residence_type': [1],
        'smoking_status': [2],
        'bmi_category': [1],
        'age_category': [2],
        'glucose_level_category': [1]
    })

def test_prediction_combination(sample_input_data):
    # Simular la carga de los modelos
    xgb_model = XGBoostStrokeModel.load_model('src/model/xgboost_model.joblib', 'src/model/xgb_scaler.joblib')
    nn_model, nn_scaler = tf.keras.models.load_model('src/model/nn_stroke.keras'), joblib.load('src/model/nn_scaler.joblib')

    # Predicción del modelo XGBoost
    xgb_probabilities = xgb_model.predict_proba(sample_input_data)[0][1]
    
    # Predicción de la red neuronal
    inputs_nn = nn_scaler.transform(sample_input_data)
    nn_probabilities = nn_model.predict(inputs_nn).flatten()
    nn_probability = nn_probabilities[0]
    
    # Combinar predicciones
    combined_probability = 0.6 * xgb_probabilities + 0.4 * nn_probability
    assert 0 <= combined_probability <= 1, "La probabilidad combinada debe estar entre 0 y 1"


In [4]:
import subprocess
import pytest

def test_streamlit_app_runs():
    """Test if the Streamlit app runs without errors."""
    # Ejecutar el script de la aplicación con subprocess
    result = subprocess.run(["streamlit", "run", "GUI.py", "--headless"], capture_output=True, text=True)

    # Verificar si el proceso fue exitoso
    assert result.returncode == 0, f"La aplicación de Streamlit falló con el siguiente error: {result.stderr}"
    assert "Running on" in result.stdout, "No se encontró la salida esperada en la aplicación"

